In [1]:
import keras
from keras.models import Model
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout,BatchNormalization,Input
import tensorflow as tf
class MyDence(keras.layers.Layer):
    def __init__(self,num_outputs):
        super(MyDence,self).__init__()
        self.out=num_outputs
    def get_config(self):
        cfg = super().get_config()
        return cfg

    def build(self,input_shape):
        self.kernel=self.add_weight(shape=[self.out,input_shape[-1]],initializer='glorot_normal', trainable=True,)
        # self.kernel=tf.Variable(tf.random.normal([self.out,input_shape[-1]]),i)

    def call(self,input):
        xw=tf.matmul(input,tf.transpose(self.kernel))
        w_norm = tf.norm(self.kernel, axis = 1) + 1e-8
        logits = xw/w_norm
        return logits
layer=MyDence(25)

num_category = 25

inp=Input((35,35,1))
x=Conv2D(filters=64,kernel_size=(5,5),activation="relu",)(inp)
x=MaxPool2D( pool_size=(1,1))(x)


x=Conv2D(filters=128,kernel_size=(5,5),activation="relu",)(x)
x=MaxPool2D( pool_size=(2,2))(x)



x=Conv2D(filters=256,kernel_size=(7,7),activation="relu",)(x)
x=MaxPool2D( pool_size=(2,2))(x)




x=Flatten()(x)

x=Dense(128,activation="relu",)(x)

x1=Dense(25,use_bias=False,)(x)
x=layer(x1)

model=Model(inp,[x,x1])
model.summary()
model.summary()


model.load_weights("C:\\Users\\PRATHMESH\\Desktop\\dce\\hand_sighv2.h5")

Using TensorFlow backend.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 35, 35, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 31, 31, 64)        1664      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 128)       204928    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 256)         1605888   
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 3, 3, 256)         0   

In [6]:
# model.save("model_final_hand_v3.h5")

In [4]:
import pyautogui as p
import math

import cv2
import time
import numpy as np
# from keras.models import load_model
# model1 = load_model('model_final_hand_v3.h5')
# test=[]
font = cv2.FONT_HERSHEY_SIMPLEX
cap = cv2.VideoCapture(cv2.CAP_DSHOW)
while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    cv2.putText(frame, "Place region of the hand inside box ",(200, 90), font, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
    
    cv2.rectangle(frame, (0, 100), (200, 300), (0, 0, 255), 2)
    
    box = frame[100:300,0:200]
    cv2.imshow("box",box)
    resize_img = cv2.resize(cv2.cvtColor(box, cv2.COLOR_BGR2GRAY ) , (35,35))
    cv2.imshow("resize_img",resize_img)
    logits = model.predict(resize_img[np.newaxis,...,np.newaxis])  
    clas=np.argmax(tf.nn.softmax(logits).numpy()[0][0])
    cv2.putText(frame, str(clas),(200, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)
    if clas==5  :
                p.press("space")
                cv2.putText(frame, "video paused",(200, 350), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)
    

    cv2.imshow("frame",frame)
    key=cv2.waitKey(1)
    if key == 27:
            cv2.destroyAllWindows() 
            cap.release()
            break
  